#### Define Imports

In [1]:
#! pip install flask_session

In [2]:
from flask import Flask, request, jsonify, render_template, session
import pandas as pd
from flask_session import Session
import sys
sys.path.insert(0,'../')
from environment import env
from model_functions.gpt_model_functions import *
config = env.env()

In [3]:
app = Flask(__name__)

# Configure server-side session
app.config['SESSION_TYPE'] = 'filesystem'
app.config['SECRET_KEY'] = 'supersecretkey'
Session(app)

@app.route('/reset', methods=['POST'])
def reset():
    session.clear()
    return jsonify({"response": "Session reset successful."})

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    # *Chat GPT assisted with session logic
    user_input = request.json.get("message")
    if not user_input:
        return jsonify({"response": "Please provide a message."})

    if 'stage' not in session:
        session['stage'] = 'initial'

    if session['stage'] == 'initial':
        # Initial stage: Handle general queries
        output, session['stage'], session['data'] = handle_initial_query(user_input)
        
    elif session['stage'] == 'amenities':
        # Handle amenity category selection
        output, session['stage'] = handle_amenity_selection(user_input, session['data'])
    else:
        output = "I specialize only in queries related to amenities, events, alerts, park fees, park locations, and park descriptions. Please clarify your question."
        session['stage'] = 'initial'

    return jsonify({"response": output})

def handle_initial_query(user_input):
    # Simulate API call to get responses for the 'amenities' endpoint
    try: 
        output = api_call(user_input)
        if isinstance(output, pd.DataFrame):
            # The nps_model_function returns a dataframe for questions that are predicted to be amenities.
            responses_df = output
            # List the amenity categories for the user to review
            categories = sorted(list(set(element.lower() for sublist in responses_df['categories'] for element in sublist)))
            if len(categories) > 0:
                # Count of categories and list them
                response = f'There are {len(categories)} amenity categories. Choose one of the following categories to learn more: '
                for category in categories:
                    response += f'\n {category}'

                return response, 'amenities', responses_df
            else:
                # If there are no categories then return this:
                return f'There are no amenities available to share at this time', 'initial', None
        return output, 'initial', None
    # If the bot does not understand the question, ask to clarify.
    except:
        return "I specialize only in queries related to amenities, events, alerts, park fees, park locations, and park descriptions. Please clarify your question.", 'initial', None

def handle_amenity_selection(user_input, responses_df):
    # Create list of amenities in the specified category
    amenities_df = responses_df[responses_df['categories'].apply(lambda x: user_input.lower() in [item.lower() for item in x])]
    amenities = sorted(list(set(name for name in amenities_df['name'])))
    
    if len(amenities) > 0:
        # Count of amenities and list them
        output = f'There {"is" if len(amenities) == 1 else "are"} {len(amenities)} {"amenity" if len(amenities) == 1 else "amenities"} in the {user_input.title()} category: '
        for amenity in amenities:
            output += f'\n {amenity}'
        return output, 'initial'
    else:
        # If there are no amenity then return this:
        return "The specified category is not an option. Please try again.", 'initial'

if __name__ == "__main__":
    app.run(port=8000)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [24/Jul/2024 08:37:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 08:38:08] "POST /chat HTTP/1.1" 200 -
